In [1]:
for p in ("Knet","ArgParse","PlyIO","Plots","JLD")
    Pkg.installed(p) == nothing && Pkg.add(p)
end
using PlyIO
using Knet
using ArgParse
using JLD
using Plots



In [2]:
###EXTRA FUNC

function preprocess(dataset, a)
    r = length(dataset)
    c = length(dataset[1])
    datasetmat = zeros(r, c)
    for i=1:r
        for j=1:c
            datasetmat[i,j] = dataset[i][j]
        end
    end
    minarray = minimum(datasetmat,1)
    maxarray = maximum(datasetmat,1)
    denom = maxarray.-minarray
    denom = denom + 1e-6 * (denom.==0)

    for i=1:r
        for j=1:c
            dataset[i][j] = 2 * a * ((dataset[i][j]-minarray[j]) / denom[j]) - a
        end
    end 

    return dataset
end

##LEAKY RELU
function lrelu(x, alpha=0.2)
    pos = max(0,x)
    neg = min(0,x) * alpha
    return pos + neg
end

###EUC DIST, Nearest Neighbor
function nn(x, A)
    smallestdist = norm(x-Array{Float64,2}(A[1]))
    smallestidx = 1
    for i in 1:length(A)
        vec=Array{Float64,2}(A[i])
        if norm(x-vec)<smallestdist
            smallestdist = norm(x-vec)
            smallestidx = i
        end
    end
    closestmodel = A[smallestidx]
    return smallestidx
end

nn (generic function with 1 method)

In [3]:
#####TRIAL

path = "scapecomp/"
##trial
data0 = load_ply("scapecomp/mesh000.ply")
#atype = gpu() >= 0 ? KnetArray{Float32} : Array{Float32}
datax = data0["vertex"]["x"]
datay = data0["vertex"]["y"]
dataz = data0["vertex"]["z"]
data1 = hcat(datax,datay)
data1 = hcat(data1,dataz)


files = readdir(path)
#println(files[1])
dataset=Any[]
for i in files
    data0 = load_ply(path * i)
    datax = data0["vertex"]["x"]
    datay = data0["vertex"]["y"]
    dataz = data0["vertex"]["z"]
    data1 = hcat(datax,datay)
    data1 = hcat(data1,dataz)
    data1 = reshape(data1,37500,1)
    dataset = push!(dataset,data1)
end

#println(size(dataset))
#dataset = minibatch(dataset, 1; xtype=atype)
#println(length(dataset))
#println(dataset)
##end of trial

In [4]:
#####NETWORK

const F = Float64

function encode(ϕ, x)
    x = mat(x)
    x = lrelu.(ϕ[1]*x .+ ϕ[2])
    μ = ϕ[3]*x .+ ϕ[4]
    logσ² = ϕ[5]*x .+ ϕ[6]
    return μ, logσ²
end

function decode(θ, z)
    z = lrelu.(θ[1]*z .+ θ[2])
    return tanh.(θ[3]*z .+ θ[4])
end

function binary_cross_entropy(x, x̂)
    s = @. x * log(x̂ + F(1e-10)) + (1-x) * log(1 - x̂ + F(1e-10))
    return -mean(s)
end

function output(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)

    #KL =  -sum(@. 1 + logσ² - μ*μ - σ²) / 2
    # Normalise by same number of elements as in reconstruction
    #KL /= M*28*28 ###1

    z = μ .+ randn!(similar(μ)) .* σ
    x̂ = decode(θ, z)
    #BCE = binary_cross_entropy(mat(x), x̂) ####2

    return x̂
end

function loss(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)

    KL =  -sum(@. 1 + logσ² - μ*μ - σ²) / 2
    # Normalise by same number of elements as in reconstruction
    #KL /= M*28*28 ###1

    #z = μ .+ randn!(similar(μ)) .* σ
    sigmaobj = ones(128,1)
    sigmaobj[1,1] = 0.1
    sigmaobj[2,1] = 0.1
    sigmaobj[3,1] = 0.5
    sigmaobj[4,1] = 0.5
    z = μ .+ KnetArray{Float64,2}(sigmaobj) .* σ
    x̂ = decode(θ, z)
    BCE = mean(abs2,mat(x)-x̂)
    #BCE = binary_cross_entropy(mat(x), x̂) ####2

    return 500*BCE + KL/500
end

function z_out(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)
    z = μ .+ randn!(similar(μ)) .* σ
    return z
end

function mseloss(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)

    z = μ .+ randn!(similar(μ)) .* σ
    x̂ = decode(θ, z)
    BCE = mean(abs2,mat(x)-x̂)
    #BCE = binary_cross_entropy(mat(x), x̂) ####2

    return BCE
end

function KLloss(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)

    KL =  -sum(@. 1 + logσ² - μ*μ - σ²) / 2
    # Normalise by same number of elements as in reconstruction
    #KL /= M*28*28 ###1

    return KL
end

function aveloss(θ, ϕ, data) #####3
    ls = F(0)
    nθ = length(θ)
    #for (x, y) in data
    for x in data
        ls += loss([θ; ϕ], x, nθ)
    end
    return ls / length(data)
end

function train!(θ, ϕ, data, opt; epochs=1)
    w = [θ; ϕ]
    for epoch=1:epochs
        for (x, y) in data
            dw = grad(loss)(w, x, length(θ))
            update!(w, dw, opt)
        end
    end
    return θ, ϕ
end

function weights(nz, nh; atype=Array{F})
    θ = [  # z->x
        xavier(nh, nz),
        zeros(nh),
        xavier(12500*3, nh), #x
        zeros(12500*3)
        ]
    θ = map(a->convert(atype,a), θ)

    ϕ = [ # x->z
        xavier(nh, 12500*3),
        zeros(nh),
        xavier(nz, nh), #μ
        zeros(nz),
        xavier(nz, nh), #σ
        zeros(nz)
        ]
    ϕ = map(a->convert(atype,a), ϕ)

    return θ, ϕ
end




weights (generic function with 1 method)

In [ ]:
function main(args="")
    s = ArgParseSettings()
    s.description="Variational Auto Encoder on MNIST dataset."
    s.exc_handler=ArgParse.debug_handler
    @add_arg_table s begin
        ("--seed"; arg_type=Int; default=-1; help="random number seed: use a nonnegative int for repeatable results")
        ("--batchsize"; arg_type=Int; default=100; help="minibatch size")
        ("--epochs"; arg_type=Int; default=10; help="number of epochs for training")
        ("--nh"; arg_type=Int; default=300; help="hidden layer dimension")
        ("--nz"; arg_type=Int; default=128; help="encoding dimention")
        ("--lr"; arg_type=Float64; default=1e-5; help="learning rate")
        ("--atype"; default=(gpu()>=0 ? "KnetArray{F}" : "Array{F}"); help="array type: Array for cpu, KnetArray for gpu")
        ("--infotime"; arg_type=Int; default=2; help="report every infotime epochs")
    end
    isa(args, String) && (args=split(args))
    if in("--help", args) || in("-h", args)
        ArgParse.show_help(s; exit_when_done=false)
        return
    end
    o = parse_args(args, s; as_symbols=true)

    atype = eval(parse(o[:atype]))
    info("using ", atype)
    o[:seed] > 0 && setseed(o[:seed])

    θ, ϕ = weights(o[:nz], o[:nh], atype=atype)
    w = [θ; ϕ]
    opt = optimizers(w, Adam, lr=o[:lr])

    path = "scapecomp/"
    files = readdir(path)
    dataset=Any[]
    for i in files
        data0 = load_ply(path * i)
        datax = data0["vertex"]["x"]
        datay = data0["vertex"]["y"]
        dataz = data0["vertex"]["z"]
        data1 = hcat(datax,datay)
        data1 = hcat(data1,dataz)
        data1=data1/2
        data1[:,2]=data1[:,2]+0.5
        data1[:,3]=data1[:,3]+0.5 
        data1 = reshape(data1,37500,1)
        dataset = push!(dataset,atype(data1))
    end
    
    
    randomout = output(w, atype(reshape(dataset[1],37500,1)), length(θ))
    #return reshape(randomout,12500,3)

    #####TRAINING PART#######
    report(epoch) = begin
            #dtrn = minibatch(xtrn, ytrn, o[:batchsize]; xtype=atype)
            #dtst = minibatch(xtst, ytst, o[:batchsize]; xtype=atype)
            println((:epoch, epoch,
                     :trn, aveloss(θ, ϕ, dataset),
                     :length, length(dataset)))
    end

    report(0); tic()
    ls_list = []
    lsTest_list = Any[]
    zlist = Any[]
    
    @time for epoch=1:o[:epochs]
        ls = 0
        kldiv = 0
        for x in dataset      
            ls += mseloss(w, atype(reshape(x,37500,1)), length(θ))
            kldiv += KLloss(w, atype(reshape(x,37500,1)), length(θ))
        end
        
        
        for x in dataset      
            dw = grad(loss)(w, atype(reshape(x,37500,1)), length(θ))
            update!(w, dw, opt)
        end               
        
        println(kldiv/length(dataset))
        println(ls/length(dataset))
        push!(ls_list, ls/length(dataset))
        (epoch % o[:infotime] == 0) && (report(epoch); toc(); tic())
        #println(23)
    end; toq()
    for x in dataset      
            z = z_out(w, atype(reshape(x,37500,1)), length(θ))
            push!(zlist,z)
        end
    
    samplein = dataset[1]
    sampleout = output(w, atype(reshape(dataset[1],37500,1)), length(θ))
    sampleout = reshape(sampleout,12500,3)
    return θ, ϕ, reshape(randomout,12500,3), w, sampleout, samplein, ls_list, zlist

end

#return randomout

#end # module

p1, p2, randomout, w, sampleout, samplein, ls_list, lsTest_list, zlist = main("--infotime 1 --seed 1 --epochs 400 --lr 1e-5");

gpu()

INFO: using Knet.KnetArray{Float64,N} where N


(:epoch, 0, :trn, 127.69787206734452, :length, 71)
4.337210470101301
0.2555320858953587
(:epoch, 1, :trn, 19.28231827665177, :length, 71)
elapsed time: 7.989938038 seconds
3410.9340828445124
0.1349417454767954
(:epoch, 2, :trn, 8.451194983653119, :length, 71)
elapsed time: 4.656464143 seconds
1805.2811392579156
0.0766874904735504
(:epoch, 3, :trn, 7.033223193866939, :length, 71)
elapsed time: 4.712257001 seconds
1279.7252534556453
0.05962257614513058
(:epoch, 4, :trn, 6.329180093339395, :length, 71)
elapsed time: 4.734207201 seconds
967.3945186538187
0.06238631937924164
(:epoch, 5, :trn, 5.9020811874461385, :length, 71)
elapsed time: 4.695611907 seconds
770.1362251454194
0.05946015481317298
(:epoch, 6, :trn, 5.619403611837242, :length, 71)
elapsed time: 4.743240431 seconds
637.0592678988979
0.058478862643145675
(:epoch, 7, :trn, 5.418368376620691, :length, 71)
elapsed time: 4.776765097 seconds
540.9423753411878
0.06207018042405687
(:epoch, 8, :trn, 5.269244933148555, :length, 71)
elaps

101.58422447849527
0.08554301938644425
(:epoch, 67, :trn, 1.8395297844248109, :length, 71)
elapsed time: 4.75850907 seconds
97.7210653616271
0.08412029777677706
(:epoch, 68, :trn, 1.7985757730111296, :length, 71)
elapsed time: 4.746640534 seconds
94.10896071341402
0.09228798589666723
(:epoch, 69, :trn, 1.7774625926888026, :length, 71)
elapsed time: 4.791609578 seconds
91.69236652115538
0.08928850103234612
(:epoch, 70, :trn, 1.7660928771107496, :length, 71)
elapsed time: 4.81461914 seconds
89.49851048593113
0.0895539708228828
(:epoch, 71, :trn, 1.7572613927424778, :length, 71)
elapsed time: 4.750445195 seconds
88.11466776672678
0.09389837370417471
(:epoch, 72, :trn, 1.7453185441784222, :length, 71)
elapsed time: 4.759060967 seconds
86.92211742653873
0.09417075313012623
(:epoch, 73, :trn, 1.7302488230600075, :length, 71)
elapsed time: 4.777433992 seconds
86.07612271749375
0.09402527844301811
(:epoch, 74, :trn, 1.7159763972601516, :length, 71)
elapsed time: 4.771451049 seconds
85.32831763

(:epoch, 133, :trn, 0.8301877214752972, :length, 71)
elapsed time: 4.752321366 seconds
67.75341653455341
0.1366931096798736
(:epoch, 134, :trn, 0.828632939871617, :length, 71)
elapsed time: 4.764833578 seconds
67.12966591908786
0.12209984925239213
(:epoch, 135, :trn, 0.8160532401588876, :length, 71)
elapsed time: 4.757856216 seconds
66.68349858242219
0.13426911606493827
(:epoch, 136, :trn, 0.806371057147978, :length, 71)
elapsed time: 4.776383936 seconds
66.10513125030788
0.12573633721599856
(:epoch, 137, :trn, 0.8005284765888476, :length, 71)
elapsed time: 4.748496414 seconds
65.82466113098772
0.1308785711736601
(:epoch, 138, :trn, 0.7913996418934138, :length, 71)
elapsed time: 4.762299191 seconds
65.21260566506415
0.1336144506789324
(:epoch, 139, :trn, 0.7917845021958335, :length, 71)
elapsed time: 4.744421694 seconds
65.46676244817928
0.13382655726602063
(:epoch, 140, :trn, 0.7834878109192652, :length, 71)
elapsed time: 4.779262605 seconds
65.10423492357373
0.13655912619231092
(:epo

51.98357538095888
0.13860554124959204
(:epoch, 199, :trn, 0.5761592955297347, :length, 71)
elapsed time: 4.795387256 seconds
51.759775360152446
0.13384216034998775
(:epoch, 200, :trn, 0.578850882894561, :length, 71)
elapsed time: 4.747658776 seconds
51.86607469822659
0.14413597716055254
(:epoch, 201, :trn, 0.5722014655683577, :length, 71)
elapsed time: 4.757331045 seconds
51.58262052479648
0.13502959897409256
(:epoch, 202, :trn, 0.5738346843155094, :length, 71)
elapsed time: 4.758623394 seconds
51.42350367091899
0.1324938987418389
(:epoch, 203, :trn, 0.567856909183357, :length, 71)
elapsed time: 4.752679898 seconds
51.37966326575543
0.12923411490850276
(:epoch, 204, :trn, 0.5731547418102658, :length, 71)
elapsed time: 4.762088632 seconds
51.24405567754539
0.13494366535716976
(:epoch, 205, :trn, 0.5653145609689716, :length, 71)
elapsed time: 4.755005187 seconds
50.980860303864645
0.13222862431679816
(:epoch, 206, :trn, 0.5652931548410722, :length, 71)
elapsed time: 4.76768396 seconds
50

43.95543104513135
0.16052348714342266
(:epoch, 265, :trn, 0.482988407021882, :length, 71)
elapsed time: 4.749920326 seconds
43.97209221203278
0.16254433156069623
(:epoch, 266, :trn, 0.46780108604280685, :length, 71)
elapsed time: 4.746536415 seconds
43.692165099309975
0.15374994639460748
(:epoch, 267, :trn, 0.46937807430419476, :length, 71)
elapsed time: 4.763915226 seconds
43.5639376387939
0.15865708176093282
(:epoch, 268, :trn, 0.4806810376643726, :length, 71)
elapsed time: 4.787433688 seconds
43.72051903061067
0.1645579311364405
(:epoch, 269, :trn, 0.4772326654022108, :length, 71)
elapsed time: 4.771769776 seconds
43.39914578282488
0.16469558140574542
(:epoch, 270, :trn, 0.46508243155799944, :length, 71)
elapsed time: 4.775333585 seconds
43.46067644676814
0.16661223848383377
(:epoch, 271, :trn, 0.4798212736505476, :length, 71)
elapsed time: 4.782954125 seconds
43.396314546979646
0.1618295754932119
(:epoch, 272, :trn, 0.47468386838848253, :length, 71)
elapsed time: 4.771941212 second

38.16707935982838
0.17597010632194257
(:epoch, 331, :trn, 0.34589895583580105, :length, 71)
elapsed time: 4.670698172 seconds
38.15772227582989
0.16781507127083162
(:epoch, 332, :trn, 0.35113904195752593, :length, 71)
elapsed time: 4.778134647 seconds
37.86827248653616
0.16994439369841738
(:epoch, 333, :trn, 0.3404906141830515, :length, 71)
elapsed time: 4.675232881 seconds
37.63152153260383
0.1663850079707725
(:epoch, 334, :trn, 0.3530252847131157, :length, 71)
elapsed time: 4.696104383 seconds
37.511141559779375
0.17721962111498482
(:epoch, 335, :trn, 0.3389708308801738, :length, 71)
elapsed time: 4.68528378 seconds
37.20543694779622
0.16716585283754884
(:epoch, 336, :trn, 0.36304685628787875, :length, 71)
elapsed time: 4.804388104 seconds
37.25487191454163
0.16772500718330702
(:epoch, 337, :trn, 0.3739009986360894, :length, 71)
elapsed time: 4.677175575 seconds
36.94183503111945
0.17282355080628348
(:epoch, 338, :trn, 0.39392519102568235, :length, 71)
elapsed time: 4.713274868 secon

35.16647817541872
0.18448134581799325


In [ ]:
println(size(randomout))
sampleout=Array{Float64,2}(sampleout)
writedlm("test5.xyz", sampleout)
#println(randomout)

samplein=reshape(samplein,12500,3)
samplein=Array{Float64,2}(samplein)
println(maximum(samplein[:,1]), minimum(samplein[:,1]))
println(maximum(samplein[:,2]), minimum(samplein[:,2]))
maximum(samplein[:,3]), minimum(samplein[:,3])
writedlm("samplein.xyz", samplein)
#dataset[1]

In [ ]:
x1 = Array{F}(zlist[1])
y1 = Array{F}(zlist[50])
scatter(x1,y1,reuse=false)

In [ ]:
plot(ls_list)
plot!(lsTest_list)


In [ ]:
ls_list

In [ ]:
atype = KnetArray{F}
path = "scapecomp/"
    files = readdir(path)
    dataset=Any[]
    for i in files
        data0 = load_ply(path * i)
        datax = data0["vertex"]["x"]
        datay = data0["vertex"]["y"]
        dataz = data0["vertex"]["z"]
        data1 = hcat(datax,datay)
        data1 = hcat(data1,dataz)
        data1=data1/2
        data1[:,2]=data1[:,2]+0.5
        data1[:,3]=data1[:,3]+0.5 
        data1 = reshape(data1,37500,1)
        dataset = push!(dataset,atype(data1))
    end
samplein = dataset[1]
thet, yy = weights(128, 300, atype=atype)
for i in 1:length(dataset)
    sample = dataset[i]
    sampleout = output(w, atype(reshape(dataset[i],37500,1)), length(thet))
    sampleout = reshape(sampleout,12500,3)
    sampleout=Array{Float64,2}(sampleout)
    writedlm("outputDL/meshout"*string(i)*".xyz", sampleout)
    writedlm("outputDL/meshin"*string(i)*".xyz", reshape(Array{Float64,2}(sample),12500,3))
end

In [ ]:
zlistFloat = zeros(Float64,length(zlist),length(zlist[1]))
for i in 1:length(zlist)
    for j in 1:length(zlist[1])
        zlistFloat[i,j] = zlist[i][j]
    end
end
zlistFloat[:,1]
x1 = Array{F}(zlistFloat[:,5])
y1 = Array{F}(zlistFloat[:,128])
scatter(x1,y1,reuse=false)



In [ ]:
nθ = length(thet)
θ, ϕ = w[1:nθ], w[nθ+1:end]
z2out = decode(θ, zlist[37])
z2out = reshape(z2out,12500,3)
z2out=Array{Float64,2}(z2out)
writedlm("z2out1.xyz", z2out)
z2out = decode(θ, zlist[13])
z2out = reshape(z2out,12500,3)
z2out=Array{Float64,2}(z2out)
writedlm("z2out2.xyz", z2out)
z2out = decode(θ, (zlist[1]+zlist[28])/2)
z2out = reshape(z2out,12500,3)
z2out=Array{Float64,2}(z2out)
writedlm("nearest2.xyz", z2out)

In [ ]:
#p1, p2, randomout, w, sampleout, samplein, ls_list, lsTest_list, zlist

save("model_sigma_scape.jld", "w", w, "zlist", zlist, "ls_list", ls_list, "lsTest_list", lsTest_list)


In [ ]:
nn(reshape(z2out,37500,1), dataset)
nearestdata1 = reshape(dataset[13],12500,3)
nearestdata1 = Array{Float64,2}(nearestdata1)
writedlm("nearestdata1.xyz", nearestdata1)


In [ ]:
#Pkg.add("Meshes")
using Meshes
#mesh_0149.obj
m = load("mesh_0149.obj")
save("my3dmodel_now_a.ply", m)
data0 = load_ply("my3dmodel_now_a.ply")
datax = data0["vertex"]["x"]
datay = data0["vertex"]["y"]
dataz = data0["vertex"]["z"]
data1 = hcat(datax,datay)
data1 = hcat(data1,dataz)